In [1]:
import dolfinx
import ufl
from petsc4py import PETSc
from mpi4py import MPI
import gmsh
import pyvista
import numpy as np

In [2]:
L = 0.5         # [m]
W = 0.1         # [m] 
nu = 0.3
EMod = 2e11     # [N/m2]
rho = 7850      # [kg/m3]

F = 10000        # [N]

lambda_ = nu/(1-2*nu)*1/(1+nu)*EMod
mu = 1/2*1/(1+nu)*EMod

In [3]:
domain = dolfinx.mesh.create_box(MPI.COMM_WORLD, [np.array([0,0,0]), np.array([L,W,W])], [50,10,10], cell_type=dolfinx.mesh.CellType.hexahedron)
V = dolfinx.fem.VectorFunctionSpace(domain, ("CG", 1))

In [4]:
def floatingBearingGetLoc(x):
    xBoolean = np.isclose(x[0], 0)
    zBoolean = np.isclose(x[2], 0)
    return xBoolean & zBoolean

floatingBearingLoc = dolfinx.mesh.locate_entities_boundary(domain, 1, floatingBearingGetLoc)
floatingBearingDof = dolfinx.fem.locate_dofs_topological(V.sub(2), 1, floatingBearingLoc)

floatingBearing = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), floatingBearingDof, V.sub(2))

In [5]:
def symmetryXGetLoc(x):
    return np.isclose(x[0], L)

symmetryXLoc = dolfinx.mesh.locate_entities_boundary(domain, 2, symmetryXGetLoc)
symmetryXDof = dolfinx.fem.locate_dofs_topological(V.sub(0), 2, symmetryXLoc)

symmetryX = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), symmetryXDof, V.sub(0))

In [6]:
def symmetryYGetLoc(x):
    return np.isclose(x[1], 0)

symmetryYLoc = dolfinx.mesh.locate_entities_boundary(domain, 2, symmetryYGetLoc)
symmetryYDof = dolfinx.fem.locate_dofs_topological(V.sub(1), 2, symmetryYLoc)

symmetryY = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), symmetryYDof, V.sub(1))

In [7]:
def loadGetLoc(x):
    # xBoolean = np.isclose(x[0], L)
    return np.isclose(x[2], W)
    # return xBoolean & zBoolean

loadLoc = dolfinx.mesh.locate_entities_boundary(domain, 2, loadGetLoc)
mt = dolfinx.mesh.meshtags(domain, 2, loadLoc, 1)
ds = ufl.Measure("ds", domain=domain, subdomain_data=mt, subdomain_id=1)
# dx = ufl.Measure("dx", domain=domain)

In [8]:
T = dolfinx.fem.Constant(domain, PETSc.ScalarType((0,0,-F/(L*W))))
# T = dolfinx.Constant(domain, ufl.as_vector((0,0,-1000)))

In [9]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2*mu*epsilon(u)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# f = dolfinx.fem.Constant(domain, PETSc.ScalarType((0,0,-9.81*rho)))
f = dolfinx.fem.Constant(domain, PETSc.ScalarType((0,0,0)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds
# L = f* v * dx + T* v * ds

In [10]:
problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[floatingBearing, symmetryX, symmetryY], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [12]:
import pyvista
pyvista.start_xvfb()
pyvista.global_theme.font.color = 'black'

# Create plotter and pyvista grid
p = pyvista.Plotter()
p.background_color = "white"

topology, cell_types, geometry = dolfinx.plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
edge = pyvista.UnstructuredGrid(np.array([8, 0, 1, 3, 2, 4, 5, 7, 6], dtype=np.int32), 
                                np.array([12]), 
                                np.array([[ 6.93889390e-18,  1.38777878e-18,  4.16333634e-18],
                                    [ 5.00000000e-01,  9.71445147e-18,  6.93889390e-18],
                                    [ 4.85722573e-17,  1.00000000e-01,  6.93889390e-18],
                                    [ 5.00000000e-01,  1.00000000e-01, -1.80411242e-17],
                                    [ 4.85722573e-17,  6.93889390e-18,  1.00000000e-01],
                                    [ 5.00000000e-01, -1.80411242e-17,  1.00000000e-01],
                                    [-1.04083409e-16,  1.00000000e-01,  1.00000000e-01],
                                    [ 5.00000000e-01,  1.00000000e-01,  1.00000000e-01]]))

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(edge, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=160)
actor_1 = p.add_mesh(warped, show_edges=False)
_ = p.add_axes(interactive=True, color='black')
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   figure_as_array = p.screenshot("deflection.png")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)